<h2>Grid Search</h2>

<ul>
<p size="12px">The following code is a grid search algorithm. The first part of the document is taken directly from the MIE377_Project_main.ipynb as it
extracts/ strcutres the <br>inputs in the same way. The second Component of this document provides a function that passes in different parameters and 
outputs the final sharp ratio. <br>The third cell / component of this document iterats through 1,140 different combinations of hyperparamters. <br><br>
For each combination of hyperparameters, the model is trained under 6 different calibration times in the data set, and then tested on the remaining data<br>from that point forward.<br><br>
The average among these 6 training and validation/test sets is stored and the grid search is complete.</p>
</ul>

In [1]:
#The following code is taken from MIE377_Project_Main

import time
import math
from scipy.stats import gmean
import matplotlib.pyplot as plt
from services.project_function import *
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

adjClose = pd.read_csv("MIE377_AssetPrices.csv", index_col=0)
factorRet = pd.read_csv("MIE377_FactorReturns.csv", index_col=0)



adjClose.index = pd.to_datetime(adjClose.index)
factorRet.index = pd.to_datetime(factorRet.index)

# Initial budget to invest ($100,000)
initialVal = 100000

# Length of investment period (in months)
investPeriod = 6

# divide the factor returns by  100
factorRet = factorRet/100

#rf and factor returns
riskFree = factorRet['RF']
factorRet = factorRet.loc[:,factorRet.columns != 'RF'];


#Identify the tickers and the dates
tickers = adjClose.columns
dates   = factorRet.index


# Calculate the stocks monthly excess returns
# pct change and drop the first null observation
returns = adjClose.pct_change(1).iloc[1:, :]

returns = returns  - np.diag(riskFree.values) @ np.ones_like(returns.values)
# Align the price table to the asset and factor returns tables by discarding the first observation.
adjClose = adjClose.iloc[1:,:]

assert adjClose.index[0] == returns.index[0]
assert adjClose.index[0] == factorRet.index[0]

In [2]:
'''
The following function takes in Hyper-Paramters from the Grid Search:
    1. Time Period
        This is the time at which calibration will occur. Example, if time_period == 5,
        then the model will have the first 5 years of data avalbale to calibrate and train
        with. This can be though of as the training data.
    2. Num_Periods
        This is the number of months (from the available in the training set) that will be used
        to actually train with. For example, if Num_Periods == 30, then from the avalaible training set,
        only the most recent 30 months will be used for training / fitting the model.
    3. Alpha
        This is the confidence level for constructing the uncertainty set in the Robust MVO.
    4. Lamda
        This is the lamda or penalty term for the l2 norm, in robust MVO.
'''

# Once again, a lot of this code is taken from the MIE377_Project_Main

def Grid_Search(time_period, num_periods, alpha, lamda, status = "grid"):


    # Start of out-of-sample test period
    testStart = returns.index[0] + pd.offsets.DateOffset(years=time_period)

    #End of the first investment period
    testEnd = testStart + pd.offsets.DateOffset(months=investPeriod) -  pd.offsets.DateOffset(days = 1)

    # End of calibration period
    calEnd = testStart -  pd.offsets.DateOffset(days = 1)

    # Total number of investment periods
    if(status == "grid"):

        NoPeriods = 10
    else:
        NoPeriods = math.ceil((returns.index[-1].to_period('M') - testStart.to_period('M')).n / investPeriod)
        
    # Number of assets
    n  = len(tickers)

    # Preallocate space for the portfolio weights (x0 will be used to calculate
    # the turnover rate)
    x  = np.zeros([n, NoPeriods])
    x0 = np.zeros([n, NoPeriods])

    # Preallocate space for the portfolio per period value and turnover
    currentVal = np.zeros([NoPeriods, 1])
    turnover   = np.zeros([NoPeriods, 1])

    #Initiate counter for the number of observations per investment period
    toDay = 0

    # Measure runtime: start the clock
    start_time = time.time()

    # Empty list to measure the value of the portfolio over the period
    portfValue = []


    for t in range(NoPeriods):

        # Subset the returns and factor returns corresponding to the current calibration period.
        periodReturns = returns[returns.index <= calEnd]
        periodFactRet = factorRet[factorRet.index <= calEnd]

        current_price_idx = (calEnd - pd.offsets.DateOffset(months=1) <= adjClose.index)&(adjClose.index <= calEnd)
        currentPrices = adjClose[current_price_idx]

        # Subset the prices corresponding to the current out-of-sample test period.
        periodPrices_idx = (testStart <= adjClose.index)&(adjClose.index <= testEnd)
        periodPrices = adjClose[periodPrices_idx]

        assert len(periodPrices) == investPeriod
        assert len(currentPrices) == 1
        # Set the initial value of the portfolio or update the portfolio value
        if t == 0:
            currentVal[0] = initialVal
        else:
            currentVal[t] = currentPrices @  NoShares.values.T
            #Store the current asset weights (before optimization takes place)
            x0[:,t] = currentPrices.values*NoShares.values/currentVal[t]

        #----------------------------------------------------------------------
        # Portfolio optimization
        # You must write code your own algorithmic trading function
        #----------------------------------------------------------------------

        x[:,t] = grid_search_function(periodReturns, periodFactRet, x0[:,t], num_periods, alpha, lamda)


        #Calculate the turnover rate
        if t > 0:
            turnover[t] = np.sum( np.abs( x[:,t] - x0[:,t] ) )

        # Number of shares your portfolio holds per stock
        NoShares = x[:,t]*currentVal[t]/currentPrices

        # Update counter for the number of observations per investment period
        fromDay = toDay
        toDay   = toDay + len(periodPrices)

        # Weekly portfolio value during the out-of-sample window
        portfValue.append(periodPrices@ NoShares.values.T)

        # Update your calibration and out-of-sample test periods
        testStart = testStart + pd.offsets.DateOffset(months=investPeriod)
        testEnd   = testStart + pd.offsets.DateOffset(months=investPeriod) - pd.offsets.DateOffset(days=1)
        calEnd    = testStart - pd.offsets.DateOffset(days=1)

    portfValue = pd.concat(portfValue, axis = 0)
    end_time = time.time()

    #--------------------------------------------------------------------------
    # 3.1 Calculate the portfolio average return, standard deviation, Sharpe ratio and average turnover.
    #-----------------------------------------------------------------------
    # Calculate the observed portfolio returns
    portfRets = portfValue.pct_change(1).iloc[1:,:]

    # Calculate the portfol"io excess returns
    portfExRets = portfRets.subtract(riskFree[(riskFree.index >= portfRets.index[0])&(riskFree.index <= portfRets.index[-1])], axis = 0)

    # Calculate the portfolio Sharpe ratio
    SR = ((portfExRets + 1).apply(gmean, axis=0) - 1)/portfExRets.std()

    # Calculate the average turnover rate
    avgTurnover = np.mean(turnover[1:])

    return SR[0], avgTurnover

    #
    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # % Program End

In [ ]:
'''
This section of code iteratates through a pre-defined combination of hyper-parameters and 
takes the average of the batch of trianing/valadiation sets containing 6 sets of training and validation periods.
The average sharpe ratio is calculate among all test sets in the batch. Then the combination of paramters with the 
best overall score is output and is to be used in the final model. Again this is just a function that was used in the 
training and validation/testing steps. 

Again just to be clear, the time_periods variable is the amount of years of data that the model will have available to it to 
train/ calibrate from. Then the model is tested and the sharpe ratio is calculated using all the data past this point. (We have
ensured there is at least 3 years of data to test the sharpe ratio in any of the data splits and that the model is only allowed
to train on a max of the latest 5 years, as per compeition requiremnts)
'''


# Define the ranges for alpha, lambda, num_periods, and time_periods
alpha_values = np.linspace(0.85, 0.99, num=5)  
lambda_values = np.linspace(2, 20, num=9)    
num_periods_values = np.arange(20, 36, 6)     
time_periods = [3,4,5,6,7,8,10]           

# Initialize variables to track the best combination and its corresponding score
best_combination = None
best_average_value = -np.inf

# Iterate over all combinations of alpha, lambda, num_periods, and time_period
for alpha in alpha_values:
    for lambda_ in lambda_values:
        for num_periods in num_periods_values:
            values = []
            for time_period in time_periods:
                value, __ = Grid_Search(time_period, num_periods, alpha, lambda_)
                values.append(value)

            # Calculate the average value for this combination of parameters
            average_value = np.mean(values)

            # Check if this is the best combination so far
            if average_value > best_average_value:
                best_average_value = average_value
                best_combination = (alpha, lambda_, num_periods)

            # Print best combination after each iteration
            print(f"Best so far -> Alpha: {best_combination[0]:.3f}, Lambda: {best_combination[1]:.3f}, "
                  f"Num Periods: {best_combination[2]}, Avg Value: {best_average_value:.4f}")

# Final Output: Best combination
print("\nFinal Best Combination:")
best_alpha, best_lambda, best_num_periods = best_combination
print(f"Alpha: {best_alpha:.3f}")
print(f"Lambda: {best_lambda:.3f}")
print(f"Num Periods: {best_num_periods}")
print(f"Best Average Value: {best_average_value:.4f}")

# Run the best parameters across the extended range of time periods
final_results = {}

for time_period in [3, 4, 5, 6, 7, 8, 9, 10]:
    result, turnover = Grid_Search(time_period, best_num_periods, best_alpha, best_lambda, "Test")
    final_results[time_period] = (result, turnover)
    print(f"Time Period: {time_period}")

# Extract results and turnovers separately
results = [res[0] for res in final_results.values()]
turnovers = [res[1] for res in final_results.values()]

# Calculate and print the average result and average turnover
average_result = np.mean(results)
average_turnover = np.mean(turnovers)

print(f"\nAverage Result Across Time Periods: {average_result:.4f}")
print(f"Average Turnover Across Time Periods: {average_turnover}")


Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 20, Avg Value: 0.2745
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Peri

In [3]:
# Define the ranges for alpha, lambda, num_periods, and time_periods
alpha_values = np.linspace(0.85, 0.99, num=5)  
lambda_values = np.linspace(2, 20, num=9)    
num_periods_values = np.arange(24, 36, 6)     
time_periods = [3,4,5,6,7,8,10]           

# Initialize variables to track the best combination and its corresponding score
best_combination = None
best_average_value = -np.inf

# Iterate over all combinations of alpha, lambda, num_periods, and time_period
for alpha in alpha_values:
    for lambda_ in lambda_values:
        for num_periods in num_periods_values:
            values = []
            for time_period in time_periods:
                value, ___ = Grid_Search(time_period, num_periods, alpha, lambda_)
                values.append(value)

            # Calculate the average value for this combination of parameters
            average_value = np.mean(values)

            # Check if this is the best combination so far
            if average_value > best_average_value:
                best_average_value = average_value
                best_combination = (alpha, lambda_, num_periods)

            # Print best combination after each iteration
            print(f"Best so far -> Alpha: {best_combination[0]:.3f}, Lambda: {best_combination[1]:.3f}, "
                  f"Num Periods: {best_combination[2]}, Avg Value: {best_average_value:.4f}")

# Final Output: Best combination
print("\nFinal Best Combination:")
best_alpha, best_lambda, best_num_periods = best_combination
print(f"Alpha: {best_alpha:.3f}")
print(f"Lambda: {best_lambda:.3f}")
print(f"Num Periods: {best_num_periods}")
print(f"Best Average Value: {best_average_value:.4f}")

# Run the best parameters across the extended range of time periods
final_results = {}

for time_period in [3, 4, 5, 6, 7, 8, 9, 10]:
    result, turnover = Grid_Search(time_period, best_num_periods, best_alpha, best_lambda, "Test")
    final_results[time_period] = (result, turnover)
    print(f"Time Period: {time_period}, Result: {result:.4f}, Turnover: {turnover:.4f}")

# Extract results and turnovers separately
results = [res[0] for res in final_results.values()]
turnovers = [res[1] for res in final_results.values()]

# Calculate and print the average result and average turnover
average_result = np.mean(results)
average_turnover = np.mean(turnovers)

print(f"\nAverage Result Across Time Periods: {average_result:.4f}")
print(f"Average Turnover Across Time Periods: {average_turnover}")

Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 24, Avg Value: 0.2615
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Periods: 30, Avg Value: 0.2685
Best so far -> Alpha: 0.850, Lambda: 2.000, Num Peri